# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [36]:
# Inicializando Spark
import findspark
findspark.init('/home/macaubas/spark-3.2.1-bin-hadoop3.2')
import pyspark

from pyspark.sql import SparkSession 

spark = SparkSession.builder.appName("clustering").getOrCreate()

In [2]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler, StandardScaler

## Carregando os dados

In [3]:
hackers = spark.read.csv('hack_data.csv', header=True, inferSchema=True)

hackers.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [5]:
for e in hackers.head(3):
    print(e)
    print('\n')

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)


Row(Session_Connection_Time=20.0, Bytes Transferred=720.99, Kali_Trace_Used=0, Servers_Corrupted=3.04, Pages_Corrupted=9.0, Location='British Virgin Islands', WPM_Typing_Speed=69.08)


Row(Session_Connection_Time=31.0, Bytes Transferred=356.32, Kali_Trace_Used=1, Servers_Corrupted=3.71, Pages_Corrupted=8.0, Location='Tokelau', WPM_Typing_Speed=70.58)




## Selecionando variáveis de interesse

In [6]:
hackers.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [7]:
col_interesse = [e for e in hackers.columns if e != 'Location']

col_interesse

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [8]:
# Selecionando variaveis de interesse
hackers = hackers.select(col_interesse)

for e in hackers.head(3):
    print(e)
    print('\n')

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, WPM_Typing_Speed=72.37)


Row(Session_Connection_Time=20.0, Bytes Transferred=720.99, Kali_Trace_Used=0, Servers_Corrupted=3.04, Pages_Corrupted=9.0, WPM_Typing_Speed=69.08)


Row(Session_Connection_Time=31.0, Bytes Transferred=356.32, Kali_Trace_Used=1, Servers_Corrupted=3.71, Pages_Corrupted=8.0, WPM_Typing_Speed=70.58)




## Assembling

In [10]:
assembler = VectorAssembler(inputCols=hackers.columns,
                           outputCol='features')

hackers = assembler.transform(hackers)

hackers.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [12]:
## Selecionando apenas a coluna de características
hackers = hackers.select('features')

hackers.show()

+--------------------+
|            features|
+--------------------+
|[8.0,391.09,1.0,2...|
|[20.0,720.99,0.0,...|
|[31.0,356.32,1.0,...|
|[2.0,228.08,1.0,2...|
|[20.0,408.5,0.0,3...|
|[1.0,390.69,1.0,2...|
|[18.0,342.97,1.0,...|
|[22.0,101.61,1.0,...|
|[15.0,275.53,1.0,...|
|[12.0,424.83,1.0,...|
|[15.0,249.09,1.0,...|
|[32.0,242.48,0.0,...|
|[23.0,514.54,0.0,...|
|[9.0,284.77,0.0,3...|
|[27.0,779.25,1.0,...|
|[12.0,307.31,1.0,...|
|[21.0,355.94,1.0,...|
|[10.0,372.65,0.0,...|
|[20.0,347.23,1.0,...|
|[22.0,456.57,0.0,...|
+--------------------+
only showing top 20 rows



## Normalizando os dados

In [15]:
scaler = StandardScaler(inputCol='features',
                             outputCol='scaledFeatures')

In [17]:
scaler_model = scaler.fit(hackers)

# Preparando os dados finais
final_data = scaler_model.transform(hackers)

final_data.show()

+--------------------+--------------------+
|            features|      scaledFeatures|
+--------------------+--------------------+
|[8.0,391.09,1.0,2...|[0.56785108466505...|
|[20.0,720.99,0.0,...|[1.41962771166263...|
|[31.0,356.32,1.0,...|[2.20042295307707...|
|[2.0,228.08,1.0,2...|[0.14196277116626...|
|[20.0,408.5,0.0,3...|[1.41962771166263...|
|[1.0,390.69,1.0,2...|[0.07098138558313...|
|[18.0,342.97,1.0,...|[1.27766494049636...|
|[22.0,101.61,1.0,...|[1.56159048282889...|
|[15.0,275.53,1.0,...|[1.06472078374697...|
|[12.0,424.83,1.0,...|[0.85177662699757...|
|[15.0,249.09,1.0,...|[1.06472078374697...|
|[32.0,242.48,0.0,...|[2.27140433866020...|
|[23.0,514.54,0.0,...|[1.63257186841202...|
|[9.0,284.77,0.0,3...|[0.63883247024818...|
|[27.0,779.25,1.0,...|[1.91649741074455...|
|[12.0,307.31,1.0,...|[0.85177662699757...|
|[21.0,355.94,1.0,...|[1.49060909724576...|
|[10.0,372.65,0.0,...|[0.70981385583131...|
|[20.0,347.23,1.0,...|[1.41962771166263...|
|[22.0,456.57,0.0,...|[1.5615904

## Criando modelo

Há duas hipóteses

* Presença de 2 hackers
* Presença de 3 hackers

In [25]:
# Criando modelos
kmeans_2 = KMeans(k=2, featuresCol='scaledFeatures').setSeed(1)
kmeans_3 = KMeans(k=3, featuresCol='scaledFeatures').setSeed(1)

In [26]:
# Estimando modelos
model_2 = kmeans_2.fit(final_data)
model_3 = kmeans_3.fit(final_data)

In [27]:
# Previsões
prev_2 = model_2.transform(final_data)
prev_3 = model_3.transform(final_data)

In [28]:
# Silhueta com distância euclidiana ao quadrado
evaluator = ClusteringEvaluator()
silhueta_2 = evaluator.evaluate(prev_2)
silhueta_3 = evaluator.evaluate(prev_3)

In [29]:
print('Silhueta da distância euclidiana ao quadrado: ')
print(f"Modelo 2 hackers: {silhueta_2:.2}\nModelo 3 Hackers: {silhueta_3:.2}")

Silhueta da distância euclidiana ao quadrado: 
Modelo 2 hackers: 0.67
Modelo 3 Hackers: 0.3


In [30]:
# Verificando centros
print('Centros para modelo com 2 hackers:\n')
model_2.clusterCenters()

[array([2.99991988, 2.92319035, 1.05261534, 3.20390443, 4.51321315,
        3.28474   ]),
 array([1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
        5.26676612])]

In [32]:
print('Centros para modelo com 3 hackers:\n')
model_3.clusterCenters()

Centros para modelo com 3 hackers:



[array([3.05623261, 2.95754486, 1.99757683, 3.2079628 , 4.49941976,
        3.26738378]),
 array([1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
        5.26676612]),
 array([2.93719177, 2.88492202, 0.        , 3.19938371, 4.52857793,
        3.30407351])]

In [23]:
# Vendo classificação
prev_2.show()

+--------------------+--------------------+----------+
|            features|      scaledFeatures|prediction|
+--------------------+--------------------+----------+
|[8.0,391.09,1.0,2...|[0.56785108466505...|         1|
|[20.0,720.99,0.0,...|[1.41962771166263...|         0|
|[31.0,356.32,1.0,...|[2.20042295307707...|         1|
|[2.0,228.08,1.0,2...|[0.14196277116626...|         1|
|[20.0,408.5,0.0,3...|[1.41962771166263...|         1|
|[1.0,390.69,1.0,2...|[0.07098138558313...|         1|
|[18.0,342.97,1.0,...|[1.27766494049636...|         1|
|[22.0,101.61,1.0,...|[1.56159048282889...|         1|
|[15.0,275.53,1.0,...|[1.06472078374697...|         1|
|[12.0,424.83,1.0,...|[0.85177662699757...|         1|
|[15.0,249.09,1.0,...|[1.06472078374697...|         1|
|[32.0,242.48,0.0,...|[2.27140433866020...|         1|
|[23.0,514.54,0.0,...|[1.63257186841202...|         1|
|[9.0,284.77,0.0,3...|[0.63883247024818...|         1|
|[27.0,779.25,1.0,...|[1.91649741074455...|         0|
|[12.0,307

In [24]:
prev_3.show()

+--------------------+--------------------+----------+
|            features|      scaledFeatures|prediction|
+--------------------+--------------------+----------+
|[8.0,391.09,1.0,2...|[0.56785108466505...|         1|
|[20.0,720.99,0.0,...|[1.41962771166263...|         0|
|[31.0,356.32,1.0,...|[2.20042295307707...|         1|
|[2.0,228.08,1.0,2...|[0.14196277116626...|         1|
|[20.0,408.5,0.0,3...|[1.41962771166263...|         1|
|[1.0,390.69,1.0,2...|[0.07098138558313...|         1|
|[18.0,342.97,1.0,...|[1.27766494049636...|         1|
|[22.0,101.61,1.0,...|[1.56159048282889...|         1|
|[15.0,275.53,1.0,...|[1.06472078374697...|         1|
|[12.0,424.83,1.0,...|[0.85177662699757...|         1|
|[15.0,249.09,1.0,...|[1.06472078374697...|         1|
|[32.0,242.48,0.0,...|[2.27140433866020...|         1|
|[23.0,514.54,0.0,...|[1.63257186841202...|         0|
|[9.0,284.77,0.0,3...|[0.63883247024818...|         1|
|[27.0,779.25,1.0,...|[1.91649741074455...|         2|
|[12.0,307

## Verificando paridade de ataques - premissa da empresa

In [35]:
prev_2.select('prediction').groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [34]:
prev_3.select('prediction').groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   79|
|         0|   88|
+----------+-----+



## Conclusão

O modelo K-Means sugere que apenas 2 hackers atacaram o sistema da empresa. Isto deve-se ao fato de que ao adicionar um terceiro hacker, a shilueta se distância do valor de 1 indicando que há uma maior interseção de classificação quando comparada ao modelo com 2 hackers.

Além disso, quando confrontamos a quantidade de ataques feitos no modelo com 2 hackers, nós verificamos a paridade de ataques de 50%, confirmando a hipótese da empresa. Quando adicioanmos um 3 hacker, um dos hackers fica com maior concentração do ataque ferindo um fator observado pela empresa.